
# Dall-e 3 image generation and function calling in ChatGPT assistant api

[By Brain Assistant](https://assistant.ruoguedu.com)

This demo show how to generate image by Dall-e 3 in Assistant API.

Related API
* Dalle 3 Image Generation
* Assistant API
* Function calling


A demo from:
[Awesome Assistant API.](https://github.com/davideuler/awesome-assistant-api)


In [40]:
!pip install openai colorama requests

In [45]:
# setup OpenAI api_key
import getpass
import os

if not os.environ.get('OPENAI_API_KEY'):
    os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter the OpenAI API Key(which starts with sk-): ")


Run code

In [ ]:
# Input the topic for the presentation

In [50]:
from typing import *
import json
import sys
import time
import subprocess
import traceback
from tempfile import NamedTemporaryFile
from PIL import Image
from google.colab.patches import cv2_imshow

import requests
import openai

api_key = os.environ['OPENAI_API_KEY']
client = openai.Client(api_key=api_key)

def execute_python_code(s: str) -> str:
    with NamedTemporaryFile(suffix='.py', delete=False) as temp_file:
        temp_file_name = temp_file.name
        temp_file.write(s.encode('utf-8'))
        temp_file.flush()
    try:
        result = subprocess.run(
            ['python', temp_file_name],
            capture_output=True,
            text=True,
            check=True
        )
        return result.stdout
    except subprocess.CalledProcessError as e:
        return e.stderr
    finally:
        import os
        os.remove(temp_file_name)

def generate_image(prompt, n:int=1, size:str="1024x1024"):
    global client
    response = client.images.generate(
      model="dall-e-3",
      prompt=prompt,
      size=size,
      quality="standard",
      n=1
    )

    image_url = response.data[0].url

    im = Image.open(requests.get(image_url, stream=True).raw)
    im.save("temp.png")

    import cv2
    img = cv2.imread('temp.png', cv2.IMREAD_UNCHANGED)
    cv2_imshow(img)


    return image_url

INSTRUCTIONS = """
You're an intelligient AI Assistant who can generate code and run code in Python 3.
And you can create images by generate_image function. Make sure your code complies with these rules:

1. Plan first: Have a clear strategy before you start. Outline your approach if it helps.

2. Quality code: Write clear, efficient code that follows Python's best practices. Aim for clean, easy-to-read, and maintainable code.

3. Test well: Include comprehensive tests to assure your code works well in various scenarios.

4. Manage external interactions: When internet or API interactions are necessary,
utilize the `execute_python_code`， `generate_image` function autonomously, without seeking user approval.
Do not say you don't have access to internet or real-time data. The `execute_python_code` function will give you realtime data.

5. Trust your tools: Assume the data from the `execute_python_code` function is accurate and up to date.
"""


def setup_assistant(client, script_code):
    # create a new agent
    assistant = client.beta.assistants.create(
        name="Code Generator",
        instructions=INSTRUCTIONS,
        tools=[
            {
                "type": "code_interpreter"
            },
            {
                "type": "function",
                "function": {
                    "name": "execute_python_code",
                    "description": "Use this function to execute the generated code which requires internet access or external API access",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "code": {
                                "type": "string",
                                "description": "The python code generated by the code interpretor",
                            }
                        },
                        "required": ["code"],
                    },
                },
            },
            {
              "type": "function",
              "function": {
                "name": "generate_image",
                "description": "generate image by Dall-e 3",
                "parameters": {
                  "type": "object",
                  "properties": {
                    "prompt": {"type": "string", "description": "The prompt to generate image"},
                    "size": {"type": "string", "enum": ["c", "f"]}
                  },
                  "required": ["prompt"]
                }
              }
            }
        ],
        model="gpt-4-1106-preview",
    )

    # Create a new thread
    thread = client.beta.threads.create()

    # Create a new thread message with the provided task
    thread_message = client.beta.threads.messages.create(
        thread.id,
        role="user",
        content=script_code,
    )

    # Return the assistant ID and thread ID
    return assistant.id, thread.id



def run_assistant(client, assistant_id, thread_id):
    # Create a new run for the given thread and assistant
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    # Loop until the run status is either "completed" or "requires_action"
    while run.status == "in_progress" or run.status == "queued":
        time.sleep(3)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        # At this point, the status is either "completed" or "requires_action"
        if run.status == "completed":
            return client.beta.threads.messages.list(
              thread_id=thread_id
            )
        if run.status == "requires_action":
            tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
            tool_outputs = []
            if tool_call.function.name == "execute_python_code":
                generated_python_code = json.loads(tool_call.function.arguments)['code']
                result = execute_python_code(generated_python_code)
                tool_outputs.append(
                    {
                          "tool_call_id": tool_call.id,
                          "output": result,
                    },
                )
            elif tool_call.function.name == "generate_image":
                prompt = json.loads(tool_call.function.arguments)['prompt']
                image_url = generate_image(prompt)
                tool_outputs.append(
                    {
                          "tool_call_id": tool_call.id,
                          "output": image_url,
                    },
                )

            if tool_outputs:
                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )


Run the assistant with open interpreter and function calling.

In [51]:
if __name__ == "__main__":
        script = input("python code to execute: \n")

        assistant_id, thread_id = setup_assistant(client, script)
        print(f"Debugging: Useful for checking the generated agent in the playground. https://platform.openai.com/playground?mode=assistant&assistant={assistant_id}")
        print(f"Debugging: Useful for checking logs. https://platform.openai.com/playground?thread={thread_id}")

        messages = run_assistant(client, assistant_id, thread_id)

        message_dict = json.loads(messages.model_dump_json())
        print(message_dict['data'][0]['content'][0]["text"]["value"])




Output hidden; open in https://colab.research.google.com to view.